In [2]:
import logging
import cobra
import cobrakbase
from cobrakbase.core.model import KBaseFBAModel
from cobrakbase.core import KBaseGenome
from cobrakbase.core import KBaseBiochemMedia
from cobrakbase.core.converters import KBaseFBAModelToCobraBuilder
import memote.suite.api as memote_api
from memote.suite.reporting import ReportConfiguration

cobrakbase 0.2.1


In [12]:
kbase_api = cobrakbase.KBaseAPI('UGOG6KLAWTCYI2ASYECYHNIIFTEXGA2J', dev=True)
modelseed = cobrakbase.modelseed.from_local('../data/ModelSEEDDatabase')

load: ../data/ModelSEEDDatabase/Biochemistry/reactions.tsv
load: ../data/ModelSEEDDatabase/Biochemistry/compounds.tsv
load: ../data/ModelSEEDDatabase/Biochemistry/Structures/Unique_ModelSEED_Structures.txt
load: ../data/ModelSEEDDatabase/Biochemistry/Aliases/Unique_ModelSEED_Compound_Aliases.txt
load: ../data/ModelSEEDDatabase/Biochemistry/Aliases/Unique_ModelSEED_Reaction_Aliases.txt
load: ../data/ModelSEEDDatabase/Biochemistry/Aliases/Unique_ModelSEED_Reaction_ECs.txt


In [4]:
params = {
    'media_id' : None,
    'model_id' : 'ecoli.model',
    'workspace' : 'filipeliu:narrative_1543990426496'
}

In [5]:
kmodel_data = kbase_api.get_object(params['model_id'], params['workspace'])
fbamodel = KBaseFBAModel(kmodel_data)
builder = KBaseFBAModelToCobraBuilder(fbamodel)
if 'genome_ref' in kmodel_data:
    logging.info("Annotating model with genome information: %s", kmodel_data['genome_ref'])
    ref_data = kbase_api.get_object_info_from_ref(kmodel_data['genome_ref'])
    genome_data = kbase_api.get_object(ref_data.id, ref_data.workspace_id)
    builder.with_genome(KBaseGenome(genome_data))
    
media = None
        
if 'media_id' in params and not params['media_id'] == "" and not params['media_id'] == None:
    print("MEDIA ID", params['media_id'])
    media_data = kbase_api.get_object(params['media_id'], params['workspace'])
    media = KBaseBiochemMedia(media_data)

if media == None:
    if 'gapfillings' in kmodel_data and len(kmodel_data['gapfillings']) > 0:
        print("Pulling media from gapfilling...", kmodel_data['gapfillings'])
        ref = kmodel_data['gapfillings'][0]['media_ref']
        ref_data = kbase_api.get_object_info_from_ref(ref)
        media_data = kbase_api.get_object(ref_data.id, ref_data.workspace_id)
        media = KBaseBiochemMedia(media_data)

if not media == None:
    builder.with_media(media)

#converts to cobra model object with builder
model = builder.build()

Pulling media from gapfilling... [{'gapfill_id': 'ecoli.model.gf.0', 'id': 'gf.0', 'integrated': 1, 'integrated_solution': '0', 'media_ref': '22384/3/1'}]


In [13]:
cobrakbase.annotate_model_with_modelseed(model, modelseed)

In [14]:
#%%capture
a, results = memote_api.test_model(model, results=True, skip=['test_thermodynamics'])
config = ReportConfiguration.load()
html = memote_api.snapshot_report(results, config)

========================================================================= test session starts ==========================================================================
platform darwin -- Python 3.6.5, pytest-4.0.0, py-1.8.0, pluggy-0.13.0
rootdir: /anaconda3/lib/python3.6/site-packages/memote/suite/tests, inifile:
plugins: rerunfailures-4.2, remotedata-0.2.1, doctestplus-0.1.3, arraydiff-0.2, openfiles-0.3.0
collected 145 items

../../../../../../../../anaconda3/lib/python3.6/site-packages/memote/suite/tests/test_annotation.py ...FF.FFFFFFFFFFFFFFFFFF..F..FFF.FF..FFFFF.F....FFFF.F..F... [ 41%]
FF...                                                                                                                                                            [ 44%]
../../../../../../../../anaconda3/lib/python3.6/site-packages/memote/suite/tests/test_basic.py ......FF...........FFFF                                           [ 60%]
../../../../../../../../anaconda3/lib/python3.6/site-packages/

In [15]:
with open(params['model_id'] + ".html", 'w') as f:
    f.write(html)

In [16]:
from IPython.display import IFrame
IFrame(src=params['model_id'] + ".html", width=900, height=600)

In [1]:
#from IPython.display import HTML
#HTML(html)